<a href="https://colab.research.google.com/github/chhatarshal/MachineLearningPractice/blob/dev/colab/001_Introduction_to_NLP_Fundamentals_in_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#https://github.com/chhatarshal/MachineLearningPractice/blob/main/tensorflow/helper/helper_functions.py

Lets get our helper function file using wget. We need this file to do some common task like unzip

In [1]:
!wget https://raw.githubusercontent.com/chhatarshal/MachineLearningPractice/main/tensorflow/helper/helper_functions.py

--2021-07-11 09:34:18--  https://raw.githubusercontent.com/chhatarshal/MachineLearningPractice/main/tensorflow/helper/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0.001s  

2021-07-11 09:34:18 (8.93 MB/s) - ‘helper_functions.py’ saved [10246/10246]



Lets now import some common function from helper functions file

In [2]:
# Import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

Now we are going to get data for our NLP research and will use unzip_data function from common functions to unzip all text data

In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# Unzip data
unzip_data("nlp_getting_started.zip")

--2021-07-11 09:34:22--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.203.128, 74.125.204.128, 64.233.188.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.203.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2021-07-11 09:34:23 (109 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



Now we got data next important think we need to do is check the data visualize visualize visualize

In [4]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


I saw some NaN in keyword and location column


In [5]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42) 
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [6]:
# What does the test dataframe look like?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


Lets see how many positive and negative sample we have

In [7]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

Here we see we have 4342 records says tweet are not disaster and rest 3271 tweets are disaster

In [8]:
# How many total samples?
len(train_df), len(test_df)

(7613, 3263)

We have 7613 train data and 3263 test data

Lets now visualize some random data

In [9]:
import random

In [10]:
random_index = random.randint(0, len(train_df))

In [11]:
print(random_index)

7580


In [12]:
train_df.iloc[random_index]["text"]

"#FX #forex #trading Cramer: Iger's 3 words that wrecked Disney's stock http://t.co/7enNulLKzM"

In [13]:
for i in range(10):
  random_index = random.randint(0, len(train_df))
  train_df.iloc[random_index]["text"]
  text, target = train_df.iloc[random_index][["text", "target"]]
  print(text, '\n', target)
  print(f"\n\n")

thunder is legit 
 0



ÛÏA voluntary evacuation is being recommended at this timeÛ for Pickerel Lake cabins across highway from #Reidlake fire says MACA #NWT #YZF 
 1



@JulieChen she shouldn't. Being with them is gonna ruin her game and Vanessa is a great player 
 0



Just thought I'd let you all know...
It's probably not a good idea to plug in your hairdryer when it's wet you will be electrocuted. 
 0



Love my girlfriend 
 0



When love and hate collided part II
Lanjut dirumah...

#yagitudeh - Jake (at Rumah Cipinang) ÛÓ https://t.co/yiLt1Bb68k 
 0



Obsolete devastation from Broad with the Ball. And Root doing it with Bat in hand. Great Day #ashes2015 #ENGvAUS http://t.co/a7TJAWWtJ7 
 0



Warne Ponting shocked by Australian collapse - Yahoo Cricket India https://t.co/hsgkTeZUCN 
 0



@ACarewornHeart Have a good un fella sorry I won't be there to get annihilated with you :( 
 0



@Treyarch @DavidVonderhaar  bring back demolition and head quarters. 
 0





We have to divide data between training and validation data and sklearn is best tool to do that

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
# Use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, # use 10% of training data for validation split
                                                                            random_state=42)

In [16]:

# Check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [17]:

# Check the first 10 samples 
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 


Converting text into numbers

When dealing with a text problem, one of the first things you'll have to do before you can build a model is to convert your text to numbers.

There are a few ways to do this, namely:

Tokenziation - direct mapping of token (a token could be a word or a character) to number
Embedding - create a matrix of feature vector for each token (the size of the feature vector can be defined and this embedding can be learned)

In [31]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Use the default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary (automatically add <OOV>)
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None, # create groups of n-words?
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None, # how long do you want your sequences to be?
                                    pad_to_max_tokens=True)

In [18]:

len(train_sentences[0].split())

7

In [19]:
train_sentences[0]

'@mogacola @zamtriossu i screamed after hitting tweet'

In [20]:
# Find the average number of tokens (words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15